In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib
from matplotlib import ticker, cm

def JtoZ(zeta: complex, c=1) -> complex:
    """
    Joukowski transformation from zeta (cylinder) to z (airfoil).

    Parameters
    ----------
    zeta (complex) : coordinates in the complex plane
    c (int = 1) : radius of the cylinder (semi-chord of airfoil)
    """
    z = zeta + c**2 / zeta

    return z

def JtoZeta(z: complex, c=1) -> complex:
    """
    Inverse Joukowski transformation from z (airfoil) to zeta (cylinder).

    Parameters
    ----------
    z (complex) : coordinates in the complex plane
    c (int = 1) : radius of the cylinder (semi-chord of airfoil)
    """
    zeta = z/2 * (1 + np.sqrt(1 - 4*c**2/(z**2)))

    return zeta

def calcStreamfunction(zeta, U=1.0, alpha=(np.pi*10/180), a=1.1, xi0=-0.1, eta0=0.1):
    """
    Calculate the streamfunctions of the flow around a cylinder at an angle of attack.
    Can be mapped back to airfoil coordinates.

    Parameters
    ----------
    zeta (complex) : coordinates in the complex plane
    U (float = 1.0) : freestream velocity parameter
    alpha : angle of attack in radians
    a : map circle to ellipse (a > c)
    xi0 : horizontal shift for symmetric airfoil
    eta0 : vertical shift for circular arc airfoil
    """
    zeta0 = xi0 + eta0*1j
    zeta = zeta - zeta0

    # Calculate streamfunction for flow around a cylinder
    F = U * ( np.exp(-alpha*1j)*zeta + np.exp(alpha*1j)*a**2/zeta )

    # Add correction for circulation to move a stagnation point to the trailing edge
    gamma = 4*np.pi*U*a*np.sin(alpha)
    F = F + gamma*1j/(2*np.pi) * np.log(zeta/a)

    return np.imag(F)

def F_zeta(x):
    z = x[0] + x[1]*1j

    F = calcStreamfunction(z)

    return F

def F_xy(x):
    zeta = x[0] + x[1]*1j

    z = JtoZeta(zeta)

    F = calcStreamfunction(z)

    return F

class JFoil:
    def __init__(self, a: int, xi0=0, eta0=0):

        self.a = a
        self.xi0 = xi0
        self.eta0 = eta0

        return
    
    def gen_circle(self):
        nu = np.linspace(0, 2*np.pi, 100)
        nui = nu * 1j
        zeta = self.a * np.exp(nui) + self.xi0 + self.eta0 * 1j

        return zeta
    
    def toReal(self):
        zeta = self.gen_circle()
        z = zeta + 1 / zeta

        return z

In [ ]:
def plot_contour(
    fig, ax, X_x, X_y, f, xlim=[-2, 2], string=r"", myLocator=ticker.LogLocator()
):
    csa = ax.contourf(X_x, X_y, f, locator=myLocator)
    ax.set(xlabel="$x_1$", ylabel="$x_2$")
    ax.set_title(string)
    ax.set_box_aspect(1.0)
    ax.grid(True)
    ax.set_xlim(xlim)
    ax.set_ylim(xlim)

    cax = ax.inset_axes([1.04, 0.0, 0.05, 1.0])
    fig.colorbar(csa, ax=ax, cax=cax)

    return


def plot_contour_function(
    fig,
    ax,
    func,
    numPts=100,
    xlim=[-2, 2],
    string=r"",
    myLocator=ticker.LogLocator(),
    colorbar=False,
    fillContour=False,
    **kwargs
):
    x1 = np.linspace(xlim[0], xlim[1], numPts)
    X_x, X_y = np.meshgrid(x1, x1)

    f = np.zeros(X_x.shape)

    for i in range(numPts):
        for j in range(numPts):
            f[i, j] = func([X_x[i, j], X_y[i, j]])
    if fillContour:
        csa = ax.contourf(X_x, X_y, f, locator=myLocator, **kwargs)
    else:
        csa = ax.contour(X_x, X_y, f, locator=myLocator, **kwargs)
    
    ax.set(xlabel="$x_1$", ylabel="$x_2$")
    ax.set_title(string)
    ax.set_box_aspect(1.0)
    ax.grid(True)
    ax.set_xlim(xlim)
    ax.set_ylim(xlim)

    if colorbar:
        cax = ax.inset_axes([1.04, 0.0, 0.05, 1.0])
        fig.colorbar(csa, ax=ax, cax=cax)

    return

In [ ]:
Foil1 = JFoil(1.1, -0.1, 0.1)
zeta = Foil1.gen_circle()
z = Foil1.toReal()

fig, ax = plt.subplots()
ax.plot(np.real(zeta), np.imag(zeta))
ax.plot(np.real(z), np.imag(z))
ax.set_aspect('equal')


In [ ]:
fig, ax = plt.subplots()
ax.plot(np.real(z), np.imag(z))
plot_contour_function(
    fig,
    ax,
    F_xy,
    numPts=200,
    xlim=[-3, 3],
    myLocator=ticker.IndexLocator(base=0.5, offset=-5),
    colorbar=False,
    fillContour=False,
    string="Stream Function",
)